### En este ejercicio crearás un programa de Python que determine si un proyectil lanzado desde cierta altitud h sobre la superficie de la Tierra permanecería o no en una orbita circular dependiendo de su velocidad inicial. Luego realizarás un experimiento aleatorio para determinar la probabilidad de que un proyectil quede atrapado en orbita circular al ser lanzado a cierta altitud. 

a)  Crea una función "Fg" que calcule la fuerza gravitacional ejercida sobre el proyectil al momento de ponerse en orbita. Las entradas de la función son:  la masa del proyectil en kilogramos $m$ y la altitud a la cual desea ponerse en órbita al proyectil $h$.  

 b) Crea otra función "Fc" que calcule la fuerza centrípeta que produce el cambio de dirección de la velocidad del proyectil viajando en su orbita circular cuyas entradas sean: la masa del proyectil "m", la velocidad inicial del proyectil en m/s "v" y la altitud de la órbita "h". 

 c) Crea un diccionario D en el cual se almacenen arreglos en que se registren los datos (m,v,h) de un conjunto de 5 proyectiles cuyas etiquetas sean "masa", "velocidad" y "altitud" respectivamente. Crea dos nuevas entradas correspondientes a la fuerza gravitacional y a la fuerza centripeta de cada proyectil a partir de las funciones Fg y Fc que creaste antes. A partir de D construye un data frame de pandas, en lo que sigue manipularas los datos usando este objeto en vez del diccionario.   
 
d) Comparando los valores de las últimas columnas, introduce al data frame una nueva columna con entradas boleanas y con etiqueta "mascara" en la cual se registre con False y True dependiendo de si para cada proyectil del conjunto 
se satisface la condición de que: $$abs(Fc-Fg)<\epsilon$$ donde $\epsilon$ esta asociado con la incertidumbre de los aparatos de medición disponibles. Supongamos que la anterior condición determina si el proyectil permanece en órbita circular o no. 

 e) Utiliza la máscara para intercambiar las entradas de D['velocidad'] correspondientes a los proyectiles que no quedaron en órbita de acuerdo al resultado del inciso anterior por los valores de la velocidad  requerida para lo contrario. ¿Se cumple que todos los satelites están en órbita? ¿Porqué?

 f) Ahora vas a hacer un experimento aleatorio usando la computadora. Considera que lanzas N (al inicio escribe y prueba el programa con un número pequeño) proyectiles con m = 100 kg a diferentes valores aleatorios de la  altitud y  rapidez inicial dentro de los siguientes rangos 

$$h \in (2000,36000) \mathrm{km}$$

$$v \in (1000,10000) \mathrm{m/s}$$

  * f.1) Genera  el diccionario de datos correspondiente, con las mismas entradas que en el inciso  c) excepto la masa. 
  *  f.2) Calcula el cociente siguiente: 
  
$$P = \frac{N_{orbita}}{N}$$

  donde $N_{orbita}$ es el número de proyectiles en la muestra que quedaron en órbita. 
   
  *  f.3) Has el experimiento para diferentes valores de $N>1000$ y observa si el valor de P converge a un valor aproximadamente constante conforme N crece. En tal caso P sería aproximadamente la probabilidad de que un proyectil quede en orbita. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import scipy.constants as con

In [19]:
#creamos las funciones Fg y Fc

def Fg(m,h):
    G = con.G
    Mt = 5.9742e+24 
    R = 6378100.
    return -G*Mt*m/((R+h)**2)

def Fc(m,h,v):
    R = 6378100.
    return -m*v**2/(R+h)

def Vo(h):
    G = con.G
    Mt = 5.9742e+24 
    R = 6378100.
    return np.sqrt(G*Mt/(R+h))


V_orbit = np.vectorize(Vo)
Fg = np.vectorize(Fg)
Fc = np.vectorize(Fc)

In [20]:
D = {'masa': np.linspace(10,50,5), 'altitud': np.linspace(2e6,35e6,5),'velocidad': np.linspace(7000,9000,5)}

In [21]:
df = pd.DataFrame(D)

df

,masa,altitud,velocidad
0,10.0,2000000.0,7000.0
1,20.0,10250000.0,7500.0
2,30.0,18500000.0,8000.0
3,40.0,26750000.0,8500.0
4,50.0,35000000.0,9000.0


In [22]:
df['Fg'] = Fg(df['masa'],df['altitud'])
df['Fc'] = Fc(df['masa'],df['altitud'],df['velocidad'])
df

,masa,altitud,velocidad,Fg,Fc
0,10.0,2000000.0,7000.0,-56.806025,-58.485814
1,20.0,10250000.0,7500.0,-28.842322,-67.656557
2,30.0,18500000.0,8000.0,-19.327350,-77.176312
3,40.0,26750000.0,8500.0,-14.532904,-87.237119
4,50.0,35000000.0,9000.0,-11.644327,-97.877863


In [29]:
def mascara(Fc, Fg):
    condicion= abs(Fc-Fg)
    
    if condicion<1:
        return True
    else:
        return False
    
mascara = np.vectorize(mascara)



In [24]:
df['mascara'] = mascara(df['Fc'], df['Fg'])
df

,masa,altitud,velocidad,Fg,Fc,mascara
0,10.0,2000000.0,7000.0,-56.806025,-58.485814,True
1,20.0,10250000.0,7500.0,-28.842322,-67.656557,False
2,30.0,18500000.0,8000.0,-19.327350,-77.176312,False
3,40.0,26750000.0,8500.0,-14.532904,-87.237119,False
4,50.0,35000000.0,9000.0,-11.644327,-97.877863,False


In [25]:
#definimos una funcion que nos de la velocidad para que el proyectil este en orbita
def Vo(h):
    G = con.G
    Mt = 5.9742e+24 
    R = 6378100.
    return np.sqrt(G*Mt/(R+h))

V_orbit = np.vectorize(Vo)


In [26]:
#inciso e
#definimos la velocidad necesaria para que el proyectil este en una orbita
df['V_orbit'] = Vo(df['altitud'])
df

,masa,altitud,velocidad,Fg,Fc,mascara,V_orbit
0,10.0,2000000.0,7000.0,-56.806025,-58.485814,True,6898.743068
1,20.0,10250000.0,7500.0,-28.842322,-67.656557,False,4896.902158
2,30.0,18500000.0,8000.0,-19.327350,-77.176312,False,4003.447485
3,40.0,26750000.0,8500.0,-14.532904,-87.237119,False,3469.320932
4,50.0,35000000.0,9000.0,-11.644327,-97.877863,False,3104.255564


In [27]:
#calculamos nuevamente el valor de la fuerza centripeta con la nueva velocidad
df['Fg'] = Fg(df['masa'],df['altitud'])
df['Fc'] = Fc(df['masa'],df['altitud'],df['V_orbit'])
df

,masa,altitud,velocidad,Fg,Fc,mascara,V_orbit
0,10.0,2000000.0,7000.0,-56.806025,-56.806025,True,6898.743068
1,20.0,10250000.0,7500.0,-28.842322,-28.842322,False,4896.902158
2,30.0,18500000.0,8000.0,-19.327350,-19.327350,False,4003.447485
3,40.0,26750000.0,8500.0,-14.532904,-14.532904,False,3469.320932
4,50.0,35000000.0,9000.0,-11.644327,-11.644327,False,3104.255564


In [30]:
#calculamos de nuevo la mascara y resulta que todos los proyectiles estan en orbita y esto se debe a que se usó la velocidad de orbita
df['mascara'] = mascara(df['Fc'], df['Fg'])
df

,masa,altitud,velocidad,Fg,Fc,mascara,V_orbit
0,10.0,2000000.0,7000.0,-56.806025,-56.806025,True,6898.743068
1,20.0,10250000.0,7500.0,-28.842322,-28.842322,True,4896.902158
2,30.0,18500000.0,8000.0,-19.327350,-19.327350,True,4003.447485
3,40.0,26750000.0,8500.0,-14.532904,-14.532904,True,3469.320932
4,50.0,35000000.0,9000.0,-11.644327,-11.644327,True,3104.255564


In [91]:
#inciso f
import random

N=3000
m=100

def Fg(h):
    G = con.G
    Mt = 5.9742e+24 
    R = 6378100.
    return -G*Mt*m/((R+h)**2)

def Fc(h,v):
    R = 6378100.
    return -m*v**2/(R+h)


In [92]:
altura= np.arange(2e6,36e6)
vel_i= np.arange(1000,10000)
C = {'altitud': random.sample(list(altura), N),'velocidad': random.sample(list(vel_i), N)}
#utilizamos la funcion random.sample pues nos arroja un valor aleatorio del intervalo que definimos y podemos recalcar el numero total de numero que requerimos
#usamos list(vel_i) pues el primer valor de la funcion random sample debe ser una secuencia o un conjunto (sequence or set)

In [93]:
dc= pd.DataFrame(C)
dc

,altitud,velocidad
0,21138046.0,4464
1,23674084.0,8182
2,18624194.0,8937
3,27121728.0,3027
4,33959319.0,5882
...,...,...
2995,2934197.0,7986
2996,28967343.0,7338
2997,8491831.0,8301
2998,32776836.0,7930


In [94]:
dc['Fg'] = Fg(dc['altitud'])
dc['Fc'] = Fc(dc['altitud'],dc['velocidad'])
dc

,altitud,velocidad,Fg,Fc
0,21138046.0,4464,-52.663567,-72.420375
1,23674084.0,8182,-44.150274,63.071030
2,18624194.0,8937,-63.786058,24.115295
3,27121728.0,3027,-35.530419,-27.351570
4,33959319.0,5882,-24.505822,20.704718
...,...,...,...,...
2995,2934197.0,7986,-459.803046,-223.645391
2996,28967343.0,7338,-31.916747,-30.828786
2997,8491831.0,8301,-180.329831,114.275883
2998,32776836.0,7930,-26.008329,-50.913701


In [95]:
dc['mascara'] = mascara(dc['Fc'], dc['Fg'])
dc

,altitud,velocidad,Fg,Fc,mascara
0,21138046.0,4464,-52.663567,-72.420375,False
1,23674084.0,8182,-44.150274,63.071030,False
2,18624194.0,8937,-63.786058,24.115295,False
3,27121728.0,3027,-35.530419,-27.351570,False
4,33959319.0,5882,-24.505822,20.704718,False
...,...,...,...,...,...
2995,2934197.0,7986,-459.803046,-223.645391,False
2996,28967343.0,7338,-31.916747,-30.828786,False
2997,8491831.0,8301,-180.329831,114.275883,False
2998,32776836.0,7930,-26.008329,-50.913701,False


In [96]:
N_orbit= dc.loc[dc['mascara']==True]
N_orbit

,altitud,velocidad,Fg,Fc,mascara
149,27162298.0,3416,-35.444517,-34.791048,True
177,32723904.0,3174,-26.078791,-25.764091,True
184,24768735.0,3584,-41.101497,-41.240325,True
298,29614943.0,9858,-30.778564,-31.341663,True
304,12685913.0,4571,-109.712682,-109.599385,True
307,28499478.0,7364,-32.778784,-32.338321,True
393,30420762.0,7311,-29.445349,-28.536339,True
463,32770499.0,7270,-26.016749,-25.296504,True
528,29628866.0,3288,-30.754766,-30.024590,True
585,35097025.0,9782,-23.179821,-23.600117,True


In [98]:
N_orbit.count()

altitud      33
velocidad    33
Fg           33
Fc           33
mascara      33
dtype: int64

In [97]:
P= 33/N
P

0.011